# Exercise 1: Basic Deduplication (Record-Level)

## Learning Objectives

In this exercise, you will:
- Learn how to remove exact duplicates from a dataset
- Understand the `exact` deduplication method
- Analyze deduplication results
- See where results are saved
- Understand record-level vs file-level deduplication

## What is Record-Level Deduplication?

**Record-level deduplication** is the process of finding and removing duplicate **rows or records** within a dataset. Unlike file-level deduplication (which compares entire files), record-level deduplication compares individual data records.

### Key Concepts

1. **Record-Based Detection**: Two rows are considered duplicates if they have the **same values** in specified key columns (like `name` and `email`), even if other columns differ.

2. **Key Columns**: The columns used to identify duplicates. Common examples:
   - `name` + `email` (person identification)
   - `product_id` + `sku` (product identification)
   - `transaction_id` (unique transaction identifier)

3. **Data Quality**: Record-level deduplication improves data quality by:
   - Removing accidental duplicate entries
   - Cleaning data before analysis
   - Reducing storage requirements
   - Improving query performance

### Record-Level vs File-Level Deduplication

| Aspect | Record-Level | File-Level |
|--------|--------------|------------|
| **What it finds** | Duplicate rows/records in a dataset | Duplicate files in a storage system |
| **Comparison unit** | Individual data records (rows) | Entire files |
| **Use case** | Cleaning data tables, removing duplicate entries | Storage optimization, backup deduplication |
| **Example** | Two rows with same name/email in a CSV | Two files with identical content but different names |
| **Method** | Compare column values | Compare file content hashes |
| **Granularity** | Row-by-row comparison | File-by-file comparison |

### Real-World Examples

- **Customer Databases**: Remove duplicate customer records (same person entered multiple times)
- **E-commerce**: Remove duplicate product listings
- **Transaction Systems**: Identify and remove duplicate transactions
- **Data Warehouses**: Clean data before loading into analytics systems
- **ETL Pipelines**: Deduplicate data during extraction/transformation
- **Data Lakes**: Remove duplicate records from large datasets

### How Record-Level Deduplication Works

1. **Identify key columns** - Determine which columns uniquely identify a record (e.g., `name`, `email`)
2. **Compare records** - Check if two rows have identical values in the key columns
3. **Remove duplicates** - Keep one copy of each unique record, remove the rest
4. **Preserve data** - Typically keep the first occurrence or use a strategy (keep latest, keep most complete, etc.)

### The `exact` Method

The `exact` method is the simplest and fastest deduplication technique:
- **How it works**: Compares exact values in specified columns
- **Speed**: Very fast - uses Spark's optimized `dropDuplicates()` function
- **Use case**: When you want to remove records that are **exactly** the same
- **Limitation**: Won't catch near-duplicates (e.g., "John Smith" vs "john smith" - different case)

### Why Deduplicate Records?

- **Data Quality**: Duplicate records can skew analysis results
- **Storage Efficiency**: Reduces dataset size
- **Performance**: Smaller datasets query faster
- **Accuracy**: Ensures each entity is counted only once
- **Compliance**: Some regulations require clean, deduplicated data

### Common Deduplication Strategies

1. **Keep First**: Keep the first occurrence, remove subsequent duplicates
2. **Keep Last**: Keep the most recent occurrence (useful for time-series data)
3. **Keep Most Complete**: Keep the record with the most non-null values
4. **Merge**: Combine information from duplicate records into one complete record

## Overview

The `exact` method is the simplest and fastest deduplication technique. It removes records that are exactly the same based on specified columns (typically `name` and `email`).

In [ ]:
# Setup: Add project root to Python path
import sys
import os

# Find project root
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    project_root = os.path.dirname(current_dir)
elif os.path.exists(os.path.join(current_dir, 'deduplicate_spark.py')):
    project_root = current_dir
else:
    # Search up directories
    test_dir = current_dir
    for _ in range(5):
        if os.path.exists(os.path.join(test_dir, 'deduplicate_spark.py')):
            project_root = test_dir
            break
        parent = os.path.dirname(test_dir)
        if parent == test_dir:
            break
        test_dir = parent
    project_root = project_root or current_dir

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✓ Added to Python path: {project_root}")

# Change to project root for file operations
os.chdir(project_root)
print(f"✓ Changed working directory to: {project_root}")


In [ ]:
from deduplicate_spark import create_spark_session, process_file_spark

# Create Spark session
spark = create_spark_session("Exercise1_BasicDeduplication")
print("✓ Spark session created")

## Step 2: Generate or Load Data

If you haven't generated data yet, uncomment and run the cell below. Otherwise, we'll use existing data.

In [ ]:
import subprocess
import os

# Generate data if it doesn't exist
data_file = os.path.join(project_root, "data", "exercise1.csv")
data_dir = os.path.join(project_root, "data")

if not os.path.exists(data_file):
    # Create data directory if needed
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print(f"✓ Created data directory: {data_dir}")
    
    print("Generating sample data...")
    script_path = os.path.join(project_root, "generate_dataset.py")
    result = subprocess.run(
        ["python", script_path, "1000", data_file],
        cwd=project_root,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("✓ Data generated successfully")
    else:
        print(f"✗ Error: {result.stderr}")
        print(f"  Script path: {script_path}")
        print(f"  Script exists: {os.path.exists(script_path)}")
else:
    print("✓ Using existing data file")

## Step 3: Run Deduplication

Now let's run the exact deduplication method.

In [ ]:
# Run exact deduplication
stats = process_file_spark(
    spark,
    os.path.join(project_root, "data", "exercise1.csv"),
    output_dir=None,  # Uses /tmp/results in Cloudera, data/ locally
    method='exact'
)

if stats:
    print(f"\nOriginal records: {stats['original_count']:,}")
    print(f"Unique records: {stats['unique_count']:,}")
    print(f"Duplicates removed: {stats['duplicates_removed']:,}")
    print(f"Deduplication rate: {stats['deduplication_rate']:.2f}%")

## Understanding the Results

After running the deduplication, you should see:
- **Original count**: Total number of records in the input file
- **Unique count**: Number of unique records after deduplication
- **Duplicates removed**: How many duplicate records were found and removed
- **Deduplication rate**: Percentage of records that were duplicates

### How to Interpret the Results

1. **Deduplication rate**: If you see 25%, that means 1 in 4 records was a duplicate
2. **Unique records**: These are the records you'll keep - one for each unique combination of key columns
3. **Output location**: Results are saved to `/tmp/results/` in Cloudera (or `data/` locally) as a Parquet file

### Example Scenario

If you have 1000 records and the deduplication shows:
- Original: 1,000 records
- Unique: 750 records
- Removed: 250 duplicates
- Rate: 25%

This means:
- 250 records were exact duplicates of other records
- You now have 750 unique records
- You saved 25% of storage space (if storing the deduplicated version)

## Questions to Answer

1. **How many duplicates were found?** Look at the "Duplicates removed" value
2. **What percentage of records were duplicates?** Check the deduplication rate
3. **Where are the results saved?** The output path will be shown in the results
4. **Why might records be duplicated?** Common causes:
   - Data entry errors (same person entered twice)
   - Multiple data sources merged without deduplication
   - System errors creating duplicate transactions
   - Data migration issues
5. **What happens if two records have the same name but different emails?** 
   - They are **NOT** considered duplicates (different key column values)
   - Only records with identical values in ALL key columns are duplicates

## Key Takeaways

- **Record-level deduplication** removes duplicate rows from datasets
- **Key columns** determine what makes a record unique
- **Exact method** is fastest but only catches exact duplicates
- **Use cases**: Data cleaning, quality improvement, storage optimization
- **Trade-off**: Fast exact deduplication vs slower fuzzy matching (covered in later exercises)

In [ ]:
# Cleanup
spark.stop()
print("✓ Spark session stopped")